In [12]:
import fitz
import os
import glob




in_path = '/home/vimalkumar/Documents/zenith/pdf'
out_path = '/home/vimalkumar/Documents/zenith/Images'



file_name_count = 1000
for file in os.listdir(in_path):

    if file.endswith('.pdf')or file.endswith('.PDF'):
        print(file)
        file_name = os.path.splitext(file)[0]
        os.makedirs(os.path.join(out_path,file_name))

        doc = fitz.open(os.path.join(in_path,file))
        print(len(doc))

        for i in range(len(doc)):
        #doc = fitz.open(in_path,file)
            page = doc.load_page(i) # number of page
            pix = page.get_pixmap(matrix=fitz.Matrix(300/72,300/72))# for 150dpi
            filename = str(i+file_name_count + 1)

            pix.pil_save(os.path.join(out_path,file_name,filename+'.jpeg'))

            file_name_count += 1000

00000001.PDF
38


In [13]:
import pytesseract
import matplotlib.pyplot as plt

In [18]:
im = cv2.imread('/home/vimalkumar/Documents/zenith/Images/00000001/3003.jpeg',0)
text = pytesseract.image_to_string(im,config = '--psm 6')
print(text)

CO: O1 RUN, BB/O02/04 #UNIONOI* MEAT PENSION PAGE 9

a GERLAND‘’S FOOD FAIR, INC MEAT CUTTERS UNION LOCAL #408 HRS PER PERIOD 946 00 RATE $119. 000 @
FROM ©O41/01/88 TO: 01/31/68
5S NAME EMP SEX BIRTHDAY HIREDATE HOURS AMOUNT STATUS DATE
433-53-8050 KEITH WILSON 457-983 M 11/06/68 11/16/67 500. 36 119 Oc
456-27-6117 HOBERT WITTE i6-920H M O6/17/64 06/10/87 199. ag 119. OO

* 441-/2-6e3' JOHN 7 YOUNG 59-966 -M 11/26/64 06/16/87 100. OO 11% OO &
452°-73-3415 TODD P YOUNG /9-946 O39/29/59 le /O3/8/7 2. eo OO TERM 66/01/08

® COMPANY TOTALS STORE- 15388. 50-TOTAL HOURS 9,756. OO-TOTAL BOLLARS 5-NEW EMPLOYEES 11-TERMINATED 1 10-ALL - t



In [30]:
import cv2
import pytesseract
from pytesseract import Output
import pandas as pd

image = cv2.imread('/home/vimalkumar/Documents/zenith/Images/00000001/26026.jpeg')
result = image.copy()
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Remove horizontal lines
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40,1))
remove_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
cnts = cv2.findContours(remove_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    cv2.drawContours(result, [c], -1, (255,255,255), 5)

# Remove vertical lines
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,40))
remove_vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
cnts = cv2.findContours(remove_vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    cv2.drawContours(result, [c], -1, (255,255,255), 5)
#plt.figure(figsize=(15,15))
#plt.imshow(result)

#img = cv2.imread('/home/vimalkumar.s/Documents/backup/2nd page_jpeg/04152020/0430556000.jpeg')
img = cv2.resize(result, (int(result.shape[1] + (result.shape[1] * .1)),
                       int(result.shape[0] + (result.shape[0] * .25))),
                 interpolation=cv2.INTER_AREA)

img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

custom_config = r'-l eng --oem 3 --psm 6 -c tessedit_char_whitelist="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-:.$%./@& *"'
d = pytesseract.image_to_data(img_rgb, config=custom_config, output_type=Output.DICT)
df = pd.DataFrame(d)

df1 = df[(df.conf != '-1') & (df.text != ' ') & (df.text != '')]
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

sorted_blocks = df1.groupby('block_num').first().sort_values('top').index.tolist()
for block in sorted_blocks:
    curr = df1[df1['block_num'] == block]
    sel = curr[curr.text.str.len() > 3]
    # sel = curr
    char_w = (sel.width / sel.text.str.len()).mean()
    prev_par, prev_line, prev_left = 0, 0, 0
    text = ''
    for ix, ln in curr.iterrows():
        # add new line when necessary
        if prev_par != ln['par_num']:
            text += '\n'
            prev_par = ln['par_num']
            prev_line = ln['line_num']
            prev_left = 0
        elif prev_line != ln['line_num']:
            text += '\n'
            prev_line = ln['line_num']
            prev_left = 0

        added = 0  # num of spaces that should be added
        if ln['left'] / char_w > prev_left + 1:
            added = int((ln['left']) / char_w) - prev_left
            text += ' ' * added
        text += ln['text'] + ' '
        prev_left += len(ln['text']) + added + 1
    text += '\n'
    print(text)
    
file = open('/home/vimalkumar/Documents/zenith/text/sample.txt', 'w')
file.write(text)
file.close()


     * 
     a                                                                                                                                                         er 
              CO: 01   RUN: 88/03/31 *UNIONO1*                   MEAT PENSION                                         PAGE    2 
    @         GERLANDS  FOOD  FAIR. INC                          MEAT CUTTERS  UNION LOCAL  *408            12S PER PERIOD.  946.00 RATE: $119. 000            & 
              FROM 02/28/88  TO: 03/31/88 
     e        ss @          NAME                            EMP    SEX BIRTHDAY   HIREDATE      4UURS     AMOUNT  STATUS    DATE                               e 
              060-30-6802   MARY K     LYNCH                77-513  F  02/27/35   07/31/82     200 00     119. 00 
              463-5i-7835   BILLY RAY  HOLT   AA            64-345  M   11/06/62  12/12/87      97.00     119.00 
   -@         459-55-6904   WILLIAM L. AUSTIN  2            64-076  M  09/29/65   02/17/88      96.75  

In [25]:
import re
with open("/home/vimalkumar/Documents/zenith/text/sample.txt", "r") as a_file:
    for line in a_file:
        print(line)



            i                                                                                                 . 

     &                                                                                                                                                                 @ 

                                                                                                                                                                       * 

          CO: O1    RUN: 88702/04  *UNIONOI*                    MEAT PENSION                                            PAGE    a 

     oe   GERLANDS   FOOD  FAIR.  INC                           MEAT CUTTERS   UNION LOCAL  @406&            HRS PER  PERTOD   96.00   RATE $119. OOO                  & 

          FROM: 01/01/86  TO: 01/31/86i 

          55 @          NAME                              EMP     SEX  BIRTHDAY   HIREDATE      HOURS      AMOUNT  STATUS     DATE                                         l 

     *    466-397-1123

In [5]:
from PIL import Image
import matplotlib.pyplot as plt
import time
import cv2
import fitz
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

start = time.time()

in_path = '/home/vimalkumar/Documents/zenith/pdf'
out_path = '/home/vimalkumar/Documents/zenith/Images'
text_path = '/home/vimalkumar/Documents/zenith/text'


for file in os.listdir(in_path):
    #print(file)
    file_name = os.path.splitext(file)[0]
    
    print(file_name)
    
    
    if file.endswith('.pdf') or file.endswith('.PDF'):
        print(file)
        
        os.makedirs(os.path.join(out_path,file_name))
        os.makedirs(os.path.join(text_path,file_name))

        doc = fitz.open(os.path.join(in_path,file))
        #print(colored('No of Pages : ','green', attrs=['bold']),len(doc))
        #print('No of Pages : ',len(doc))
        mat = fitz.Matrix(150 / 72, 150 / 72)
        for k in range(len(doc)):
            page = doc.load_page(k)  # number of page
            pix = page.get_pixmap(matrix=mat)
            #print(type(pix))
            #plt.imshow(pix)

            pix.pil_save(os.path.join(out_path,file_name,file_name+'_'+str(k+1)+'.jpeg'))


        
end = time.time()
print(f"Runtime of the program is {end - start}")
        

00000001
00000001.PDF
Runtime of the program is 14.123377799987793
